In [ ]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Purchase Prediction with AutoML Tables

<table align="left">
  <td>
    <a href="https://cloud.google.com/ml-engine/docs/tensorflow/getting-started-keras">
      <img src="https://cloud.google.com/_static/images/cloud/icons/favicons/onecloud/super_cloud.png"
           alt="Google Cloud logo" width="32px"> Read on cloud.google.com
    </a>
  </td>
  <td>
    <a href="">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

One of the most common use cases in Marketing is to predict the likelihood of conversion. Conversion could be defined by the marketer as taking a certain action like making a purchase, signing up for a free trial, subscribing to a newsletter, etc. Knowing the likelihood that a marketing lead or prospect will ‘convert’ can enable the marketer to target the lead with the right marketing campaign. This could take the form of remarketing, targeted email campaigns, online offers or other treatments.

Here we demonstrate how you can use Bigquery and AutoML Tables to build a supervised binary classification model for purchase prediction.

### Dataset

The model uses a real dataset from the [Google Merchandise store](https://www.googlemerchandisestore.com) consisting of Google Analytics web sessions.

The goal here is to predict the likelihood of a web visitor visiting the online Google Merchandise Store making a purchase on the website during that Google Analytics session. Past web interactions of the user on the store website in addition to information like browser details and geography are used to make this prediction.

This is framed as a binary classification model, to label a user during a session as either true (makes a purchase) or false (does not make a purchase). Dataset Details The dataset consists of a set of tables corresponding to Google Analytics sessions being tracked on the Google Merchandise Store. Each table is a single day of GA sessions. More details around the schema can be seen here.

You can access the data on BigQuery [here](https://support.google.com/analytics/answer/3437719?hl=en&ref_topic=3416089).

## Instructions

Here is a list of things to do with AutoML Tables:


 * Set up your local development environment (optional)
 * Set Project ID and Compute Region
 * Authenticate your GCP account
 * Import Python API SDK and create a Client instance,
 * Create a dataset instance and import the data.
 * Create a model instance and train the model.
 * Evaluating the trained model.
 * Deploy the model on the cloud for online predictions.
 * Make online predictions.
 * Undeploy the model



# 1. Before you begin

## Project setup

Follow the [AutoML Tables documentation](https://cloud.google.com/automl-tables/docs/) to:
* Create a Google Cloud Platform (GCP) project, replace "your-project" with your GCP project ID and set local development environment.
* Enable billing.
* Enable AutoML API.
* Enter your project ID in the cell below. Then run the  cell to make sure the

**If you are using Colab or AI Platform Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step from the AutoML Tables documentation

Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands

In [ ]:
PROJECT_ID = "<your-project>" # @param {type:"string"}
COMPUTE_REGION = "us-central1" # Currently only supported region.
! gcloud config set project $PROJECT_ID



---



This section runs initialization and authentication. It creates an authenticated session which is required for running any of the following sections.

## Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the GCP Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. From the **Service account** drop-down list, select **New service account**.

3. In the **Service account name** field, enter a name.

4. From the **Role** drop-down list, select
   **AutoML > AutoML Admin** and
   **Storage > Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:    
  from google.colab import files
  keyfile_upload = files.upload()
  keyfile = list(keyfile_upload.keys())[0]
  %env GOOGLE_APPLICATION_CREDENTIALS $keyfile
# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS /path/to/service_account.json

## Install the client library
Run the following cell.

In [ ]:
%pip install --quiet google-cloud-automl

### Import libraries and define constants

First, import Python libraries required for training,
The code example below demonstrates importing the AutoML Python API module into a python script. 

In [ ]:
# AutoML library
from google.cloud import automl_v1beta1 as automl

import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types
import matplotlib.pyplot as plt

Restart the kernel to allow automl_v1beta1 to be imported for Jupyter Notebooks.

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Create API Client to AutoML Service*

**If you are using AI Platform Notebooks**, or *Colab* environment is already
authenticated using GOOGLE_APPLICATION_CREDENTIALS. Run this step.

In [ ]:
client = automl.TablesClient(project=PROJECT_ID, region=COMPUTE_REGION)

**If you are using Colab or Jupyter**, and you have defined a service account
follow the following steps to create the AutoML client

You can see a different way to create the API Clients using service account.

In [ ]:
# from google.oauth2 import service_account
# credentials = service_account.Credentials.from_service_account_file('/path/to/service_account.json')
# client = automl.TablesClient(project=PROJECT_ID, region=COMPUTE_REGION, credentials=credentials)

---

## Storage setup

You also need to upload your data into [Google Cloud Storage](https://cloud.google.com/storage/) (GCS) or [BigQuery](https://cloud.google.com/bigquery/). 
For example, to use GCS as your data source:

* [Create a GCS bucket](https://cloud.google.com/storage/docs/creating-buckets).
* Upload the training and batch prediction files.



---



# 3. Import, clean, transform and perform feature engineering on the training Data

### Create dataset in AutoML Tables


Select a dataset display name and pass your table source information to create a new dataset.


In [ ]:
#@title Create dataset { vertical-output: true, output-height: 200 }

dataset_display_name = 'colab_trial1' #@param {type: 'string'}

dataset = client.create_dataset(dataset_display_name)
dataset

Create a bucket to store the training data in

In [ ]:
#@title Create bucket to store data in { vertical-output: true, output-height: 200 }

bucket_name = '<your-bucket>' #@param {type: 'string'}


### Import Dependencies


In [ ]:
!sudo pip install google-cloud-bigquery google-cloud-storage pandas pandas-gbq gcsfs oauth2client

import datetime
import pandas as pd

import gcsfs
from google.cloud import bigquery
from google.cloud import storage

client_bq = bigquery.Client(location='US', project=PROJECT_ID)

### Transformation and Feature Engineering Functions

The data cleaning and transformation step was by far the most involved. It includes a few sections that create an AutoML tables dataset, pull the Google merchandise store data from BigQuery, transform the data, and save it multiple times to csv files in google cloud storage.

The dataset that is made viewable in the AutoML Tables UI. It will eventually hold the training data after that training data is cleaned and transformed.

This dataset has only around 1% of its values with a positive label value of True i.e. cases when a transaction was made. This is a class imbalance problem. There are several ways to handle class imbalance. We chose to oversample the positive class by random over sampling. This resulted in an artificial increase in the sessions with the positive label of true transaction value.

There were also many columns with either all missing or all constant values. These columns would not add any signal to our model, so we dropped them.

There were also columns with NaN rather than 0 values. For instance, rather than having a count of 0, a column might have a null value. So we added code to change some of these null values to 0, specifically in our target column, in which null values were not allowed by AutoML Tables. However, AutoML Tables can handle null values for the features.

#### Feature Engineering

The dataset had rich information on customer location and behavior; however, it can be improved by performing feature engineering. Moreover, there was a concern about data leakage. The decision to do feature engineering, therefore, had two contributing motivations: remove data leakage without too much loss of useful data, and to improve the signal in our data.

##### Weekdays

The date seemed like a useful piece of information to include, as it could capture seasonal effects. Unfortunately, we only had one year of data, so seasonality on an annual scale would be difficult (read impossible) to incorporate. Fortunately, we could try and detect seasonal effects on a micro, with perhaps equally informative results. We ended up creating a new column of weekdays out of dates, to denote which day of the week the session was held on. This new feature turned out to have some useful predictive power, when added as a variable into our model.

##### Data Leakage

The marginal gain from adding a weekday feature, was overshadowed by the concern of data leakage in our training data. In the initial naive models we trained, we got outstanding results. So outstanding that we knew that something must be going on. As it turned out, quite a few features functioned as proxies for the feature we were trying to predict: meaning some of the features we conditioned on to build the model had an almost 1:1 correlation with the target feature. Intuitively, this made sense.

One feature that exhibited this behavior was the number of page views a customer made during a session. By conditioning on page views in a session, we could very reliably predict which customer sessions a purchase would be made in. At first this seems like the golden ticket, we can reliably predict whether or not a purchase is made! The catch: the full page view information can only be collected at the end of the session, by which point we would also have whether or not a transaction was made. Seen from this perspective, collecting page views at the same time as collecting the transaction information would make it pointless to predict the transaction information using the page views information, as we would already have both. One solution was to drop page views as a feature entirely. This would safely stop the data leakage, but we would lose some critically useful information. Another solution, (the one we ended up going with), was to track the page view information of all previous sessions for a given customer, and use it to inform the current session. This way, we could use the page view information, but only the information that we would have before the session even began. So we created a new column called previous_views, and populated it with the total count of all previous page views made by the customer in all previous sessions. We then deleted the page views feature, to stop the data leakage.

Our rationale for this change can be boiled down to the concise heuristic: only use the information that is available to us on the first click of the session. Applying this reasoning, we performed similar data engineering on other features which we found to be proxies for the label feature. We also refined our objective in the process: For a visit to the Google Merchandise store, what is the probability that a customer will make a purchase, and can we calculate this probability the moment the customer arrives? By clarifying the question, we both made the result more powerful/useful, and eliminated the data leakage that threatened to make the predictive power trivial.

In [ ]:
def balanceTable(table):
    #class count
  count_class_false, count_class_true = table.totalTransactionRevenue.value_counts()

    #divide by class
  table_class_false = table[table["totalTransactionRevenue"] == False]
  table_class_true = table[table["totalTransactionRevenue"] == True]

    #random over-sampling
  table_class_true_over = table_class_true.sample(count_class_false, replace = True)
  table_test_over = pd.concat([table_class_false, table_class_true_over])
  return table_test_over


def partitionTable(table, dt=20170500):
  #the automl tables model could be training on future data and implicitly learning about past data in the testing
  #dataset, this would cause data leakage. To prevent this, we are training only with the first 9 months of data (table1)
  #and doing validation with the last three months of data (table2).
  table1 = table[table["date"] <= dt]
  table2 = table[table["date"] > dt]
  return table1, table2

def N_updatePrevCount(table, new_column, old_column):
  table = table.fillna(0)
  table[new_column] = 1
  table.sort_values(by=['fullVisitorId','date'])
  table[new_column] = table.groupby(['fullVisitorId'])[old_column].apply(lambda x: x.cumsum())
  table.drop([old_column], axis = 1, inplace = True)
  return table


def N_updateDate(table):
  table['weekday'] = 1
  table['date'] = pd.to_datetime(table['date'].astype(str), format = '%Y%m%d')
  table['weekday'] = table['date'].dt.dayofweek
  return table


def change_transaction_values(table):
  table['totalTransactionRevenue'] = table['totalTransactionRevenue'].fillna(0)
  table['totalTransactionRevenue'] = table['totalTransactionRevenue'].apply(lambda x: x!=0)
  return table

def saveTable(table, csv_file_name, bucket_name):
  table.to_csv(csv_file_name, index = False)
  storage_client = storage.Client()
  bucket = storage_client.get_bucket(bucket_name)
  blob = bucket.blob(csv_file_name)
  blob.upload_from_filename(filename = csv_file_name)

### Import training data

You also have the option of just downloading the file, FULL.csv, [here](https://storage.cloud.google.com/cloud-ml-data/automl-tables/notebooks/trial_for_c4m/FULL.csv), instead of running the code below. Just be sure to move the file into the google cloud storage bucket you specified above.

In [ ]:
#@title Input name of file to save data to { vertical-output: true, output-height: 200 }
query = """
SELECT
 date, 
 device, 
 geoNetwork, 
 totals, 
 trafficSource, 
 fullVisitorId 
FROM 
 `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
 _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB('2017-08-01', INTERVAL 366 DAY)) AND
 FORMAT_DATE('%Y%m%d',DATE_SUB('2017-08-01', INTERVAL 1 DAY))
"""
df = client_bq.query(query).to_dataframe()
print(df.iloc[:3])
path_to_data_pre_transformation = "FULL.csv" #@param {type: 'string'}
saveTable(df, path_to_data_pre_transformation, bucket_name)

### Unnest the Data

In [ ]:
query = """
SELECT
 date, 
 device, 
 geoNetwork, 
 totals, 
 trafficSource, 
 fullVisitorId 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB('2017-08-01', INTERVAL 366 DAY)) AND
FORMAT_DATE('%Y%m%d',DATE_SUB('2017-08-01', INTERVAL 1 DAY))
"""
df = client_bq.query(query).to_dataframe()

In [ ]:
#some transformations on the basic dataset
#@title Input the name of file to hold the unnested data to { vertical-output: true, output-height: 200 }
unnested_file_name = "FULL_unnested.csv" #@param {type: 'string'}

You also have the option of just downloading the file, FULL_unnested.csv, [here](https://storage.cloud.google.com/cloud-ml-data/automl-tables/notebooks/trial_for_c4m/FULL_unnested.csv), instead of running the code below. Just be sure to move the file into the google cloud storage bucket you specified above.

In [ ]:

table = pd.read_csv("gs://"+bucket_name+"/"+unnested_file_name, low_memory=False)

column_names = ['device', 'geoNetwork','totals', 'trafficSource']

for name in column_names:
  print(name)
  table[name] = table[name].apply(lambda i: dict(eval(i)))
  temp = table[name].apply(pd.Series)
  table = pd.concat([table, temp], axis=1).drop(name, axis=1)

#need to drop a column
table.drop(['adwordsClickInfo'], axis = 1, inplace = True)
saveTable(table, unnested_file_name, bucket_name)

### Run the Transformations

In [ ]:
table = pd.read_csv("gs://"+bucket_name+"/"+unnested_file_name, low_memory=False)

consts = ['transactionRevenue', 'transactions', 'adContent', 'browserSize', 'campaignCode', 
'cityId', 'flashVersion', 'javaEnabled', 'language', 'latitude', 'longitude', 'mobileDeviceBranding', 
'mobileDeviceInfo', 'mobileDeviceMarketingName','mobileDeviceModel','mobileInputSelector', 'networkLocation', 
'operatingSystemVersion', 'screenColors', 'screenResolution', 'screenviews', 'sessionQualityDim', 'timeOnScreen',
'visits', 'uniqueScreenviews', 'browserVersion','referralPath','fullVisitorId', 'date']

table = N_updatePrevCount(table, 'previous_views', 'pageviews')
table = N_updatePrevCount(table, 'previous_hits', 'hits')
table = N_updatePrevCount(table, 'previous_timeOnSite', 'timeOnSite')
table = N_updatePrevCount(table, 'previous_Bounces', 'bounces')

table = change_transaction_values(table)

table1, table2 = partitionTable(table)
table1 = N_updateDate(table1)
table2 = N_updateDate(table2)
#validation_unnested_FULL.csv = the last 3 months of data

table1.drop(consts, axis = 1, inplace = True)
table2.drop(consts, axis = 1, inplace = True)

saveTable(table2,'validation_unnested_FULL.csv', bucket_name)

table1 = balanceTable(table1)

#training_unnested_FULL.csv = the first 9 months of data
saveTable(table1, 'training_unnested_balanced_FULL.csv', bucket_name)


In [ ]:
#@title ... take the data source from GCS { vertical-output: true } 

dataset_gcs_input_uris = ['gs://{}/training_unnested_balanced_FULL.csv'.format(bucket_name),] #@param
import_data_operation = client.import_data(
    dataset=dataset,
    gcs_input_uris=dataset_gcs_input_uris
)
print('Dataset import operation: {}'.format(import_data_operation))

# Synchronous check of operation status. Wait until import is done.
import_data_operation.result()
dataset = client.get_dataset(dataset_name=dataset.name)
dataset



---



# 4. Update dataset: assign a label column and enable nullable columns

AutoML Tables automatically detects your data column type. Depending on the type of your label column, AutoML Tables chooses to run a classification or regression model. If your label column contains only numerical values, but they represent categories, change your label column type to categorical by updating your schema.

In [ ]:
# List table specs
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type:

features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])
    
# Table schema pie chart.

type_counts = {}
for column_spec in column_specs.values():
  type_name = data_types.TypeCode.Name(column_spec.data_type.type_code)
  type_counts[type_name] = type_counts.get(type_name, 0) + 1
    
plt.pie(x=type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()


In [ ]:
#@title Update a column: set to not nullable { vertical-output: true }

update_column_response = client.update_column_spec(
    dataset=dataset,
    column_spec_display_name='totalTransactionRevenue',
    nullable=False,
)
update_column_response

**Tip:** You can use kwarg `type_code='CATEGORY'` in the preceding `update_column_spec(..)` call to convert the column data type from `FLOAT64` `to `CATEGORY`.

### Update dataset: assign a target column

In [ ]:
#@title Update dataset { vertical-output: true }

update_dataset_response = client.set_target_column(
    dataset=dataset,
    column_spec_display_name='totalTransactionRevenue',
)
update_dataset_response

# 5. Creating a model

### Train a model

To create the datasets for training, testing and validation, we first had to consider what kind of data we were dealing with. The data we had keeps track of all customer sessions with the Google Merchandise store over a year. AutoML tables does its own training and testing, and delivers a quite nice UI to view the results in. For the training and testing dataset then, we simply used the over sampled, balanced dataset created by the transformations described above. But we first partitioned the dataset to include the first 9 months in one table and the last 3 in another. This allowed us to train and test with an entirely different dataset that what we used to validate.

Moreover, we held off on oversampling for the validation dataset, to not bias the data that we would ultimately use to judge the success of our model.

The decision to divide the sessions along time was made to avoid the model training on future data to predict past data. (This can be avoided with a datetime variable in the dataset and by toggling a button in the UI)

Training the model may take one hour or more. The following cell keeps running until the training is done. If your Colab times out, use `client.list_models()` to check whether your model has been created. Then use model name to continue to the next steps. Run the following command to retrieve your model. Replace `model_name` with its actual value.

    model = client.get_model(model_name=model_name)
    
Note that we trained on the first 9 months of data and we validate using the last 3.

In [ ]:
#@title Create model { vertical-output: true }
#this will create a model that can be access through the auto ml tables colab
model_display_name = 'trial_1' #@param {type:'string'}

create_model_response = client.create_model(
    model_display_name,
    dataset=dataset,
    train_budget_milli_node_hours=1000,
)
print('Create model operation: {}'.format(create_model_response.operation))
# Wait until model training is done.
model = create_model_response.result()
model



---



# 6. Make a prediction

In this section, we take our validation data prediction results and plot the Precision Recall Curve and the ROC curve of both the false and true predictions.

There are two different prediction modes: online and batch. The following cell shows you how to make a batch prediction. Please replace the 'your_test_bucket' of the gcs_destination with your own bucket where the predictions results will be stored by AutoML Tables.

In [ ]:
#@title Start batch prediction { vertical-output: true, output-height: 200 }

batch_predict_gcs_input_uris = ['gs://cloud-ml-data-tables/notebooks/validation_unnested_FULL.csv',] #@param
batch_predict_gcs_output_uri_prefix = 'gs://{}'.format(bucket_name) #@param {type:'string'}
batch_predict_response = client.batch_predict(
    model=model, 
    gcs_input_uris=batch_predict_gcs_input_uris,
    gcs_output_uri_prefix=batch_predict_gcs_output_uri_prefix,
)
print('Batch prediction operation: {}'.format(batch_predict_response.operation))
# Wait until batch prediction is done.
batch_predict_result = batch_predict_response.result()
batch_predict_response.metadata

# 7. Evaluate your prediction

The follow cell creates a Precision Recall Curve and a ROC curve for both the true and false classifications.
Fill in the batch_predict_results_location with the location of the results.csv file created in the previous "Make a prediction" step


In [ ]:

import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

def invert(x):
  return 1-x

def switch_label(x):
  return(not x)
batch_predict_results_location = 'gs://<your-test-bucket>/' #@param {type:'string'}

table = pd.read_csv(batch_predict_results_location +'/<resultid>/tables_1.csv')
y = table["totalTransactionRevenue"]
scores = table["totalTransactionRevenue_1.0_score"]
scores_invert = table['totalTransactionRevenue_0.0_score']

#code for ROC curve, for true values
fpr, tpr, thresholds = metrics.roc_curve(y, scores)
roc_auc = metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for True')
plt.legend(loc="lower right")
plt.show()


#code for ROC curve, for false values
plt.figure()
lw = 2
label_invert = y.apply(switch_label)
fpr, tpr, thresholds = metrics.roc_curve(label_invert, scores_invert)
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for False')
plt.legend(loc="lower right")
plt.show()


#code for PR curve, for true values

precision, recall, thresholds = metrics.precision_recall_curve(y, scores)


plt.figure()
lw = 2
plt.plot( recall, precision, color='darkorange',
         lw=lw, label='Precision recall curve for True')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve for True')
plt.legend(loc="lower right")
plt.show()

#code for PR curve, for false values

precision, recall, thresholds = metrics.precision_recall_curve(label_invert, scores_invert)
print(precision.shape)
print(recall.shape)

plt.figure()
lw = 2
plt.plot( recall, precision, color='darkorange',
          label='Precision recall curve for False')
plt.xlim([0.0, 1.1])
plt.ylim([0.0, 1.1])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve for False')
plt.legend(loc="lower right")
plt.show()

